# Semester 1 Project Submission



## Elliot

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Ian

In [2]:
import sqlite3 as sql
import pandas as pd

### Items to be Resolved:
- check for nulls in the sql database tables being used
    - See following cell for findings.
- resolve found nulls 
    - Rows where null can be excluded. This is due to size and number of entries in dataset tables.
- experiment with joins to get tables wanted
    - Joined movie_basics with movie_ratings. Tried further joins but they get very large, very quickly (even using inner joins).
- export joined tables to dataframe
- determine how to best use data for analysis
____________

- persons % bunch of null values in birth_year, death_year, and in primary_profession
- writers % NONE ARE NULL
- directors % NONE ARE NULL
- known_for % NONE ARE NULL
- principals % NO NULLS IN CATEGORY
- movie_basics % missing data in genres, runtime_minutes
- movie_ratings % NONE ARE NULL
- movie_akas % NO TITLES ARE NULL

In [3]:
# creating connection to database
conn = sql.connect("databases/im.db")

In [27]:
# movie basics query stage 1
q = """
SELECT
    *
FROM
    movie_basics
/*where statement below reduces dataset from 146144 rows to 112233*/
WHERE
    runtime_minutes IS NOT NULL AND genres IS NOT NULL
;"""
pd.read_sql(q, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
4,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
...,...,...,...,...,...,...
112228,tt9916160,Drømmeland,Drømmeland,2019,72.0,Documentary
112229,tt9916170,The Rehearsal,O Ensaio,2019,51.0,Drama
112230,tt9916186,Illenau - die Geschichte einer ehemaligen Heil...,Illenau - die Geschichte einer ehemaligen Heil...,2017,84.0,Documentary
112231,tt9916190,Safeguard,Safeguard,2019,90.0,"Drama,Thriller"


In [26]:
# movie ratings query stage 1
q = """
SELECT
    *
FROM
    movie_ratings
/*where statement below reduces dataset from 73856 rows to 73856 NO NULLS*/
WHERE
    averagerating IS NOT NULL AND numvotes IS NOT NULL
;"""
pd.read_sql(q, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [25]:
# movie_akas query stage 1
q = """
SELECT
    *
FROM
    movie_akas
/*where statement below reduces dataset from 331703 rows to 331703 NO NULLS*/
WHERE
    title IS NOT NULL
;"""
pd.read_sql(q, conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [36]:
# persons table query stage 1
q = """
SELECT
    *
FROM
    persons
WHERE
    primary_profession IS NULL
;"""
pd.read_sql(q, conn)

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0120366,Ralph Bunche,1904.0,1971.0,None
1,nm0256636,Emperor Franz Josef,1830.0,1916.0,None
2,nm0278661,Renee Firestone,NaN,NaN,None
3,nm0441808,Mike Katz,1944.0,NaN,None
4,nm0688002,Norman Podhoretz,1930.0,NaN,None
...,...,...,...,...,...
51335,nm9964778,Brian Espitia,NaN,NaN,None
51336,nm9967439,Jade Doucette,NaN,NaN,None
51337,nm9969939,Anna Bligh,NaN,NaN,None
51338,nm9976388,Naomie Monplaisir,NaN,NaN,None


In [40]:
# principals exploration query
q = """
SELECT
    *
FROM
    principals
;"""
pd.read_sql(q, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [37]:
# directors exploration query
q = """
SELECT
    *
FROM
    directors
;"""
pd.read_sql(q, conn)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [38]:
# known_for exploration query
q = """
SELECT
    *
FROM
    known_for
;"""
pd.read_sql(q, conn)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


______________________________

### Consolidation Queries

In [35]:
# combining movie basics and movie ratings
q = """
SELECT
    mb.primary_title, mb.runtime_minutes, mb.genres, mr.averagerating
FROM
    movie_basics AS mb
    JOIN movie_ratings AS mr
        USING(movie_id)
;"""
pd.read_sql(q, conn)

,primary_title,runtime_minutes,genres,averagerating
0,Sunghursh,175.0,"Action,Crime,Drama",7.0
1,One Day Before the Rainy Season,114.0,"Biography,Drama",7.2
2,The Other Side of the Wind,122.0,Drama,6.9
3,Sabse Bada Sukh,NaN,"Comedy,Drama",6.1
4,The Wandering Soap Opera,80.0,"Comedy,Drama,Fantasy",6.5
...,...,...,...,...
73851,Diabolik sono io,75.0,Documentary,6.2
73852,Sokagin Çocuklari,98.0,"Drama,Family",8.7
73853,Albatross,NaN,Documentary,8.5
73854,La vida sense la Sara Amat,NaN,None,6.6


In [43]:
# persons to principals to movie table
q = """
SELECT 
    pe.person_id, pe.primary_name, mb.primary_title, pr.category
FROM
    persons AS pe
    JOIN principals AS pr
        USING(person_id)
    JOIN movie_basics AS mb
        USING(movie_id)
LIMIT 10
;"""
pd.read_sql(q, conn)

,person_id,primary_name,primary_title,category
0,nm0061671,Mary Ellen Bauder,Smurfs: The Lost Village,producer
1,nm0061865,Joseph Bauer,Satin,composer
2,nm0061865,Joseph Bauer,Caesar and Otto's Deadly Xmas,composer
3,nm0061865,Joseph Bauer,Elf-Man,composer
4,nm0061865,Joseph Bauer,Journey to the Forbidden Valley,composer
5,nm0061865,Joseph Bauer,Somebody Marry Me,composer
6,nm0061865,Joseph Bauer,Tooken,composer
7,nm0062070,Bruce Baum,Spinners: Izzy Lyon - The Unspun Truth,actor
8,nm0062195,Axel Baumann,The Only Real Game,cinematographer
9,nm0062195,Axel Baumann,Sky and Ground,cinematographer


## Frida

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Enrique

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## John

In [ ]:
# Your code here - remember to use markdown cells for comments as well!